# Imitation Learning

We saw that a Reinforcement Learning agent seeks to find a policy that maximizes the expected return:

$$
V^*(s) = \max_\pi V_\pi(s) = E_\pi \Big[ \sum_{t=0}^{\infty} r(S_t, A_t) | S_0 = s   \Big]
$$


This problem can be solved in different ways. If we know completely our MDP, we can use dynamic programming (course 2). If the MDP is unknown, we need to use sample-based algorithms, like Q-Learning (course 3/4). 

However, if we can simulate an optimal agent, that is, an *expert*, we can use supervised learning to train our agent using samples obtained with this expert. This is the main idea of imitation learning. 

Here you will train a agent to play the game Connect Four using imitation learning. The agent will be trained to learn $V^*(s)$ directly, and an optimal policy can be found by:

$$
\pi^*(s) \in \arg\max_a \Big[ r(s,a) + \sum_{s'}p(s'|s, a)V^*(s') \Big]
$$
    

## Connect Four

Connect Four (*puissance 4* in French) is a 2-player game where each player has discs of a given color, for instance red for player 1 and green for player 2, that need to be placed on a grid with rows and columns. The two players then alternate turns dropping one of their discs at a time into an unfilled column, until one of the players manage to connect four discs of his color. You can find an example of the gameplay in [Wikipedia](https://en.wikipedia.org/wiki/Connect_Four#Gameplay).

# 1. Training data

**Important**: if the script to download the data does not work, download the file 'c4.dat' [here](http://data.lip6.fr/cadene/rl/c4.dat) and place it in the folder data/

Consider two players, A and B, and a connect four game of width 7 and height 6. Each entry of the dataset corresponds to a sequence

$$
x_1 x_2 x_3 \ldots x_{T_1} \quad v(x)
$$

where $x = x_1 x_2 x_3 \ldots x_{T_1}$ is an integer (in base 10) such that player A dropped a disk at position $x_i$ at time $i$ and B played $x_{i+1}$ at time step $i+1$. Thus, $x$ represents a *state* of the board. The value **$v(x)$ is the outcome of the game, which can be 1 if A wins, -1 if B wins and 0 otherwise**. This value represents the best possible outcome of the game for player A by playing optimaly starting from state $x$. 

Our goal is to train a supervised learning algorithm to estimate $v(x)$ for any state $x$. $v(x)$ is converted to a target that equals 0 if A wins, 1 if B wins and 2 otherwise, so that it can be passed to a PyTorch classifier.

## Visualizing the data

The function `tensorize_board` takes an input $x_1 x_2 x_3 \ldots x_{T_1}$ and transforms it into an array of size 6x7 such that an empty cell is filled with 0, a cell that has a disc from player A is filled with +1, and -1 if it is a disc from player B. 

Remark: each digit $x_i$ is in $\{1, 2, 3, 4, 5, 6, 7\}$ and represents the column where the player chose to play. Also, for the function `tensorize_board` to work, **player B has made the last move**.

Examples:

In [1]:
from c4 import tensorize_board

print(tensorize_board(112))
print(tensorize_board(1122))
print(tensorize_board(112234))

tensor([[-1., -1., -0., -0., -0., -0., -0.],
        [ 1., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.]])
tensor([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [-1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 1.,  1.,  1., -1.,  0.,  0.,  0.],
        [-1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])


## Loading the data

In [2]:
import os
from os import path as osp
import utils
import torch

# Load data and get some parameters
train_set, test_set, args, kwargs = utils.get_train_test_data()

# pytorch loaders
train_loader = torch.utils.data.DataLoader(train_set,
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set,
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [3]:
print('number of examples in training set = ', len(train_set))
print('number of examples in test set = ', len(test_set))
print(train_set[0])
print(test_set[-1])

number of examples in training set =  3499
number of examples in test set =  1501
(tensor([[[ 0.,  1.,  1., -1.,  0., -1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]]), 0)
(tensor([[[ 1., -1.,  0.,  1.,  0.,  0., -1.],
         [ 0.,  1.,  0., -1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]]), 2)


# Exercise 1. - Defining and Training NNs in PyTorch

Now you have to define neural networks for classification using pytorch. The training code is provided below, see function `train`

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
# Just an example of CNN, you can define your own architecture!

class Net(nn.Module):

    def __init__(self, d=1, nclass=3):
        super(Net, self).__init__()
        # define the elementary blocks of your neural network
        self.features = nn.Sequential(
            nn.Conv2d(1, 6*d, kernel_size=5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(6*d),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2, padding=1),
            nn.Conv2d(6*d, 16*d, kernel_size=5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(16*d),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2, padding=1)
        )
        self.classif = nn.Sequential(
            nn.Linear(16*2*2*d, 120*d, bias=False),
            nn.BatchNorm1d(120*d),
            nn.ReLU(inplace=True),
            nn.Linear(120*d, 84*d, bias=False),
            nn.BatchNorm1d(84*d),
            nn.ReLU(inplace=True),
            nn.Linear(84*d, nclass)
        )

    def forward(self, x):
        # x= ... (x)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classif(x)
        return F.log_softmax(x, dim=1)

In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    confm = ConfusionMeter(3)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        confm.add(output.detach(), target.detach())
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    confm.show()

for batch_idx, (data, target) in enumerate(train_loader):
    print(data.shape)
    break

torch.Size([64, 1, 6, 7])


# Exercise 2. Evaluating the performance

Complete the function below and evaluate the performance of the learned model.

In [7]:
# You can check https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#test-the-network-on-the-test-data
# and adapt the code for testing whic is provided.
def test(args, model, device, test_loader):
    confm = ConfusionMeter(3)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            confm.add(output, target)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    confm.show()

In [8]:
# Training and testing the model
from confm import ConfusionMeter

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(),"c4_cnn.pt")

Train Epoch: 1 [0/3499 (0%)]	Loss: 1.151262
Train Epoch: 1 [640/3499 (18%)]	Loss: 0.950320
Train Epoch: 1 [1280/3499 (36%)]	Loss: 1.022363
Train Epoch: 1 [1920/3499 (55%)]	Loss: 0.899808
Train Epoch: 1 [2560/3499 (73%)]	Loss: 0.903462
Train Epoch: 1 [3200/3499 (91%)]	Loss: 0.890910
confusion_matrix [[2024, 139, 21], [771, 108, 14], [383, 37, 2]]
binacc_cid,92.67399267399267
binacc_cid,12.094064949608063
binacc_cid,0.47393364928909953

Test set: Average loss: 0.8839, Accuracy: 941/1501 (63%)

confusion_matrix [[776, 160, 0], [218, 165, 0], [135, 47, 0]]
binacc_cid,82.90598290598291
binacc_cid,43.08093994778068
binacc_cid,0.0
Train Epoch: 2 [0/3499 (0%)]	Loss: 0.904225
Train Epoch: 2 [640/3499 (18%)]	Loss: 0.888887
Train Epoch: 2 [1280/3499 (36%)]	Loss: 0.835539
Train Epoch: 2 [1920/3499 (55%)]	Loss: 0.755452
Train Epoch: 2 [2560/3499 (73%)]	Loss: 0.883151
Train Epoch: 2 [3200/3499 (91%)]	Loss: 0.793979
confusion_matrix [[2012, 172, 0], [690, 203, 0], [360, 62, 0]]
binacc_cid,92.12454212